In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
import os
import mne
import random
from sklearn.metrics import classification_report
from numpy import zeros, newaxis
from scipy import signal
from scipy import fftpack


Bad key "text.kerning_factor" on line 4 in
C:\Users\GC\Anaconda3\envs\ark\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
MDD_dataset_Filenames = os.listdir('./full_dataset/MDD/EC/')
H_dataset_Filenames = os.listdir('./full_dataset/H/EC/')

MDD_dataset_list = []
H_dataset_list = []

for filename in MDD_dataset_Filenames:
    data = mne.io.read_raw_edf('./full_dataset/MDD/EC/' + str(filename))
    MDD_dataset_list.append(data)  
    
for filename in H_dataset_Filenames:
    data = mne.io.read_raw_edf('./full_dataset/H/EC/' + str(filename))
    H_dataset_list.append(data)  


Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S1 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S10 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S11  EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S13 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S14 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S15 EC.edf...
EDF file detected
Setting channel info structure...
Creating ra

In [3]:
#MDD_dataset_list=MDD_dataset_list[0:29]
#H_dataset_list=H_dataset_list[0:29]
MDD_dataset_list[26].ch_names

['EEG Fp1-LE',
 'EEG F3-LE',
 'EEG C3-LE',
 'EEG P3-LE',
 'EEG O1-LE',
 'EEG F7-LE',
 'EEG T3-LE',
 'EEG T5-LE',
 'EEG Fz-LE',
 'EEG Fp2-LE',
 'EEG F4-LE',
 'EEG C4-LE',
 'EEG P4-LE',
 'EEG O2-LE',
 'EEG F8-LE',
 'EEG T4-LE',
 'EEG T6-LE',
 'EEG Cz-LE',
 'EEG Pz-LE',
 'EEG A2-A1',
 'EEG 23A-23R',
 'EEG 24A-24R']

In [4]:
MDD_dataframe = [] #MDD 환자의 데이터프레임 리스트
H_dataframe = [] #건강한 사람의 데이터프레임 리스트
y = []
for raw in MDD_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    MDD_dataframe.append(temp.iloc[:30720]) #256 * 120 = 2min
    y.append(1)
    
for raw in H_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(temp)
    H_dataframe.append(temp.iloc[:30720]) #256 * 120 = 2min
    y.append(0)
y = np.asarray(y)    
MDD_dataframe[0]

,EEG Fp1-LE,EEG F3-LE,EEG C3-LE,EEG P3-LE,EEG O1-LE,EEG F7-LE,EEG T3-LE,EEG T5-LE,EEG Fz-LE,EEG Fp2-LE,EEG F4-LE,EEG C4-LE,EEG P4-LE,EEG O2-LE,EEG F8-LE,EEG T4-LE,EEG T6-LE,EEG Cz-LE,EEG Pz-LE,EEG A2-A1
0.000000,3.850294e-06,-0.000002,-0.000002,-5.500420e-07,-1.650126e-06,-7.450568e-06,4.500343e-07,-2.450187e-06,2.750210e-06,4.550347e-06,5.000381e-08,3.550271e-06,0.000007,2.500191e-07,5.450416e-06,0.000006,0.000003,2.500191e-07,0.000003,2.450187e-06
0.003906,2.250172e-06,-0.000004,-0.000001,3.500267e-07,-1.500114e-07,-7.750591e-06,-1.150088e-06,-2.450187e-06,4.500343e-07,2.650202e-06,-2.550195e-06,3.750286e-06,0.000008,2.050156e-06,8.550652e-06,0.000007,0.000004,-5.500420e-07,0.000006,2.050156e-06
0.007812,-5.000381e-08,-0.000005,-0.000002,8.500649e-07,5.500420e-07,-5.250401e-06,-2.650202e-06,-2.250172e-06,-2.250172e-06,5.000381e-08,-4.950378e-06,2.950225e-06,0.000008,3.450263e-06,8.950683e-06,0.000008,0.000005,-1.750134e-06,0.000006,2.350179e-06
0.011719,-2.350179e-06,-0.000005,-0.000002,1.050080e-06,9.500725e-07,-2.050156e-06,-3.750286e-06,-2.050156e-06,-4.450340e-06,-2.250172e-06,-5.950454e-06,2.350179e-06,0.000007,4.250324e-06,6.750515e-06,0.000006,0.000005,-2.750210e-06,0.000005,3.250248e-06
0.015625,-3.650278e-06,-0.000005,-0.000002,9.500725e-07,1.150088e-06,-1.500114e-07,-4.550347e-06,-1.650126e-06,-5.450416e-06,-3.450263e-06,-5.350408e-06,2.450187e-06,0.000005,4.750362e-06,3.250248e-06,0.000003,0.000006,-3.150240e-06,0.000003,4.450340e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119.980469,-1.350103e-06,0.000004,0.000003,5.500420e-07,1.150088e-06,2.650202e-06,2.150164e-06,7.500572e-07,3.450263e-06,1.850141e-06,3.050233e-06,4.500343e-07,0.000002,4.150317e-06,6.350484e-06,0.000006,0.000004,1.450111e-06,0.000002,1.950149e-06
119.984375,-1.050080e-06,0.000003,0.000004,1.950149e-06,2.150164e-06,2.450187e-06,3.050233e-06,2.050156e-06,2.050156e-06,7.500572e-07,2.250172e-06,-1.650126e-06,0.000002,3.950301e-06,4.750362e-06,0.000007,0.000004,5.000381e-08,0.000003,1.650126e-06
119.988281,4.500343e-07,0.000002,0.000005,3.950301e-06,2.650202e-06,2.450187e-06,5.250401e-06,3.350256e-06,1.750134e-06,5.000381e-08,2.850217e-06,-1.250095e-06,0.000004,3.850294e-06,2.350179e-06,0.000007,0.000005,1.500114e-07,0.000005,1.050080e-06
119.992188,2.050156e-06,0.000002,0.000007,6.050462e-06,2.650202e-06,2.550195e-06,7.550576e-06,4.650355e-06,2.250172e-06,-2.500191e-07,4.250324e-06,1.550118e-06,0.000006,3.750286e-06,4.500343e-07,0.000008,0.000006,1.550118e-06,0.000007,6.500496e-07


In [5]:
#MDD[26]환자 열 개수 조정
MDD_dataframe[26] = MDD_dataframe[26][MDD_dataframe[0].columns]

In [6]:
low_pass_filter = signal.firwin(101, cutoff=32, fs=256, pass_zero='lowpass')
high_pass_filter = signal.firwin(101, cutoff=0.5, fs=256, pass_zero='highpass')

In [7]:
for j in range(0, len(MDD_dataframe)):
    col = list(MDD_dataframe[j].columns)
    for i in range(0, len(col)):
        clear_eeg = signal.lfilter(low_pass_filter, [1.0], MDD_dataframe[j][col[i]])
        clear_eeg = signal.lfilter(high_pass_filter, [1.0], clear_eeg)
        MDD_dataframe[j][col[i]] = clear_eeg

In [8]:
for j in range(0, len(H_dataframe)):
    col = list(H_dataframe[j].columns)
    for i in range(0, len(col)):
        clear_eeg = signal.lfilter(low_pass_filter, [1.0], H_dataframe[j][col[i]])
        clear_eeg = signal.lfilter(high_pass_filter, [1.0], clear_eeg)
        H_dataframe[j][col[i]] = clear_eeg

C:\Users\GC\Anaconda3\envs\ark\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
all_dataframe = MDD_dataframe + H_dataframe


ValueError: cannot copy sequence with size 30720 to array axis with dimension 20

In [9]:
delta_pass_filter =  signal.firwin(101, cutoff=[0.5,4], fs=256, pass_zero='bandpass')
theta_pass_filter =  signal.firwin(101, cutoff=[4,8], fs=256, pass_zero='bandpass')
alpha_pass_filter =  signal.firwin(101, cutoff=[8,13], fs=256, pass_zero='bandpass')
beta_pass_filter =  signal.firwin(101, cutoff=[13,30], fs=256, pass_zero='bandpass')
from sklearn.preprocessing import MinMaxScaler

def band_power(channel):
    df = pd.DataFrame()
    df['delta'] = signal.lfilter(delta_pass_filter, [1.0], channel)
    df['theta'] = signal.lfilter(theta_pass_filter, [1.0], channel)
    df['alpha'] = signal.lfilter(alpha_pass_filter, [1.0], channel)
    df['beta'] = signal.lfilter(beta_pass_filter, [1.0], channel)
    
    return df

#scaler = MinMaxScaler()
#def nomalization(df):
#    for col in df.columns:
#        scaler.fit(df)
#        df = scaler.fit_transform(df)
#    return df

In [33]:
def eeg_band_power_seperate(input dataframe)    
    X = np.zeros((1,20,4,30720))
    for df in input_dataframe:
        columns = list(df.columns)
        band_power_list = np.zeros((1,4,30720))
        #band_power_list = band_power_list[newaxis,:,:]
        for i in range(len(columns)):
            ch = band_power(df[columns[i]])
            #ch = nomalization(ch) #Min-Max
            ch = np.asarray(ch)
            band_power_list = np.vstack([band_power_list, ch.T[newaxis,:,:]])
        band_power_list = band_power_list[1:]
        X = np.vstack([X, band_power_list[newaxis,:,:,:]])
        print(X.shape)
    return X

(2, 20, 4, 30720)
(3, 20, 4, 30720)
(4, 20, 4, 30720)
(5, 20, 4, 30720)
(6, 20, 4, 30720)
(7, 20, 4, 30720)
(8, 20, 4, 30720)
(9, 20, 4, 30720)
(10, 20, 4, 30720)
(11, 20, 4, 30720)
(12, 20, 4, 30720)
(13, 20, 4, 30720)
(14, 20, 4, 30720)
(15, 20, 4, 30720)
(16, 20, 4, 30720)
(17, 20, 4, 30720)
(18, 20, 4, 30720)
(19, 20, 4, 30720)
(20, 20, 4, 30720)
(21, 20, 4, 30720)
(22, 20, 4, 30720)
(23, 20, 4, 30720)
(24, 20, 4, 30720)
(25, 20, 4, 30720)
(26, 20, 4, 30720)
(27, 20, 4, 30720)
(28, 20, 4, 30720)
(29, 20, 4, 30720)
(30, 20, 4, 30720)
(31, 20, 4, 30720)


In [37]:
from sklearn.metrics import roc_curve
from keras import models
from keras import layers
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [23]:
def index_to_list(list, index):
    return [list[i] for i in index]

In [27]:
accuracy_scores = []
prec_scores = []
rec_scores = []
f1_scores = []
import keras.backend.tensorflow_backend as Keras_GPU
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=3)
for train, test in kfold.split(all_dataframe, y):
    X_train = index_to_list(all_dataframe, train)
    y_train = y[train]
    X_test = index_to_list(all_dataframe, test)
    y_test = y[test]
    
    print(X_train)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

    

[              EEG Fp1-LE     EEG F3-LE     EEG C3-LE     EEG P3-LE  \
0.000000   -5.749518e-13  3.061432e-13  2.762756e-13  8.213598e-14   
0.003906   -1.338526e-12  1.153560e-12  6.982685e-13  9.094814e-14   
0.007812   -1.607877e-12  2.397878e-12  1.133587e-12 -7.100974e-14   
0.011719   -8.401791e-13  3.570480e-12  1.468936e-12 -3.856536e-13   
0.015625    8.879855e-13  4.121732e-12  1.613116e-12 -7.073019e-13   
...                  ...           ...           ...           ...   
119.980469  2.849867e-07 -1.107508e-06 -4.064137e-06 -2.287663e-06   
119.984375  7.197372e-07 -1.367127e-06 -4.471316e-06 -1.531693e-06   
119.988281  1.670731e-06 -1.585996e-06 -4.496257e-06 -1.465900e-06   
119.992188  2.811218e-06 -1.487880e-06 -3.775405e-06 -1.762777e-06   
119.996094  3.672498e-06 -9.136120e-07 -2.246254e-06 -1.961322e-06   

               EEG O1-LE     EEG F7-LE     EEG T3-LE     EEG T5-LE  \
0.000000    2.464079e-13  1.112569e-12 -6.720216e-14  3.658784e-13   
0.003906    4.5205

In [18]:
#CNN 모델을 사용
    model = models.Sequential()

    #1
    model.add(layers.Conv1D(64, 4,
                    border_mode='same',
                    activation='relu',
                    input_shape=(80,30720)))
    model.add(layers.Dropout(0.5))
    model.add(layers.MaxPooling1D(pool_size=2,padding = 'same'))
    
    ##-- layer 4
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    
    model.add(layers.Dense(1, activation='sigmoid'))


    model.compile(optimizer='Adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', recall_m, precision_m, f1_m])

    model.fit(X[train], y[train], epochs=30, batch_size=128, verbose=0)
    
    scores = model.evaluate(X[test], y[test], verbose=0)
    
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))
    print("\n")
    accuracy_scores.append(scores[1] * 100)
    prec_scores.append(scores[2])
    rec_scores.append(scores[3])
    f1_scores.append(scores[4])
    
print("%.2f (+/- %.2f%%)" % (np.mean(accuracy_scores), np.std(accuracy_scores)))
print("%.2f (+/- %.2f%%)" % (np.mean(prec_scores), np.std(prec_scores)))
print("%.2f (+/- %.2f%%)" % (np.mean(rec_scores), np.std(rec_scores)))
print("%.2f (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))

IndentationError: unexpected indent (<ipython-input-18-4db71db91ea3>, line 2)